In [6]:
# 必要なライブラリをインポートする。
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [7]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
# パースする。
transaction = Tx.parse(stream)
# 手数料を確認する。
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
# 秘密鍵
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
# 署名
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
#  ハッシュ対象
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
# パースする。
point = S256Point.parse(sec)
signature = Signature.parse(der)
# 署名を検証する。
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
# 署名ハッシュを取得する
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
# ハッシュ値に変換
h256 = hash256(modified_tx)
# ビッグエンディアン整数として認識される。
z = int.from_bytes(h256, 'big')
# zを取得する。
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [10]:
from ecc import S256Point, Signature
# SECフォーマットの公開鍵
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
# DERフォーマットの電子署名
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
# 署名ハッシュz
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
#　署名を検証する。
point.verify(z, signature)

True

### 練習問題 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [15]:
# 練習問題 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### 練習問題 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [20]:
# 練習問題 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.263s

OK


In [1]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
# インプットトランザクション
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
# インプットインデックス
prev_index = 13
# インプットトランザクションを用意する。
tx_in = TxIn(prev_tx, prev_index)
# 配列を宣言
tx_outs = []
change_amount = int(0.33*100000000)
# 20バイトのハッシュ値に変換する。
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
# スクリプトを取得する。
change_script = p2pkh_script(change_h160)
# アウトプットトランザクション生成
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
# 同様に2つ目のアウトプットが作られる。
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
# トランザクションを生成する。
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [8]:
# トランザクションを署名する関数
from ecc import PrivateKey
from helper import SIGHASH_ALL
# トランザクションのハッシュ値を生成する。
z = transaction.sig_hash(0)
# 秘密鍵を生成する。
private_key = PrivateKey(secret=8675309)
# DERフォーマットの署名を生成する。
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
# SECフォーマットの秘密鍵を生成する。
sec = private_key.point.sec()
# Scriptを生成する。
script_sig = Script([sig, sec])
# スクリプトシグを生成する。
transaction.tx_ins[0].script_sig = script_sig 
# トランザクションを出力する。
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


### 練習問題 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [9]:
# 練習問題 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.209s

OK


In [4]:
# アドレスを生成するためのテスト
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'harukiTest'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mjH7bSQQD8AuEaohFThhtCxa5wyHiRS38x


### 練習問題 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://live.blockcypher.com/btc/pushtx

In [19]:
# 練習問題 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
my_address = 'mmhv8u8kvRiLzmL364gMdpxnApgAB56nnQ'
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# 秘密鍵を生成する。
passphrase = b'test@test-address.com some secret only you know'
# secret = little_endian_to_int(hash256(passphrase))
secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
priv = PrivateKey(secret)
# アドレスを出力する。
print(priv.point.address(compressed = True, testnet=True))

# インプットトランザクションを取得する。
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
# トランザクションインデックスを取得する。
prev_index = 1
# インプットトランザクションを用意する。
tx_in = TxIn(prev_tx, prev_index)
# インプットトランザクション用の配列を宣言
tx_ins = []
# 配列に要素を追加する。
tx_ins.append(TxIn(prev_tx, prev_index))
# アウトプットトランザクション用の配列を宣言
tx_outs = []
# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
# 送金額を設定する。
target_amount = tx_in.value(testnet=True) * 0.6
target_satoshi = int(target_amount * 100000000)
# 手数料を算出する。
#stream = BytesIO(prev_tx)
# パースする。
#transaction2 = Tx.parse(stream)
#fee = transaction2.fee() 
# お釣りを算出する。
change_amount = 0.009
change_satoshi = int(change_amount * 100000000)

# 1つ目のアウトプットトランザクションを生成する。
target_h160 = decode_base58(target_address)
# Script_pubkeyを生成する。
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_satoshi, script_pubkey=target_script)
# 2つめのアウトプットトランザクションを生成する。
change_h160 = decode_base58(my_address)
# Script_pubkeyを生成する。
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_satoshi, script_pubkey=change_script)
# それぞれ配列に追加する。
tx_outs.append(target_output )
tx_outs.append(change_output )
#  トランザクションデータを生成する。
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
# 送信者の秘密鍵でトランザクションに署名する。
print(tx_obj.sign_input(0, priv))
# 16進数でトランザクションを出力する。
print(tx_obj.serialize().hex())

mmhv8u8kvRiLzmL364gMdpxnApgAB56nnQ
False
010000000199a24308080ab26e6fb65c4eccfadf76749bb5bfa8cb08f291320b3c21e56f0d010000006b483045022100851e17339ce65ce76eb91209f7e67cc6e802c90789deab1011ba2fa778e3555602204a018090f7982fe73f0e6deab3e8a8c49297ae792be047b49e5cf4b4611c66b201210212ea3a4481ba46b61514b3c159b608678da9bc586642418f8fbfc1294c9ec196ffffffff0200c06e31d91001001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac9fbb0d00000000001976a91443e3876d6f6fcaaeac38295ebbed8d7b4f39411888ac00000000


###  練習問題 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [21]:
# 練習問題 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
change_address = 'mjH7bSQQD8AuEaohFThhtCxa5wyHiRS38x'
# 秘密鍵を生成する。
passphrase = b'test@test-address.com some secret only you know'
# secret = little_endian_to_int(hash256(passphrase))
secret = little_endian_to_int(hash256(passphrase))
# 秘密鍵を生成する
priv = PrivateKey(secret)
priv2 = PrivateKey(secret=8675309)

# インプットトランザクション用の配列を宣言
tx_ins = []
# 1つ目のインプットトランザクションを取得する。
prev_tx = bytes.fromhex('1db91a35684f2767befa0341e85f1f26e7618950c5a023da4e133339b589a9e9')
# トランザクションインデックスを取得する。
prev_index = 1
# 1つ目のインプットトランザクションを設定する。
tx_in = TxIn(prev_tx, prev_index)
# 送金額を設定する。
target_amount = tx_in.value(testnet=True)
target_satoshi = int(target_amount * 100000000)
# 配列に要素を追加する。
tx_ins.append(TxIn(prev_tx, prev_index))
# get the prev_tx and prev_index from the transaction in Exercise 4
# 2つ目のインプットトランザクションを取得する。
prev_tx2 = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
# トランザクションインデックスを取得する。
prev_index2 = 1
# 2つ目のインプットトランザクションを設定する。
tx_in2 = TxIn(prev_tx2, prev_index2)
# 送金額を設定する。
target_amount2 = tx_in2.value(testnet=True)
target_satoshi2 = int(target_amount2 * 100000000)
# 配列に要素を追加する。
tx_ins.append(TxIn(prev_tx2, prev_index2))
# set the fee to some reasonable amount
# 手数料を算出する。
#stream = BytesIO(prev_tx)
# パースする。
#transaction2 = Tx.parse(stream)
#fee = transaction2.fee() 

# アプトプットトランザクション用の配列を生成する。
tx_outs = []
# アウトプットトランザクションを生成する。
target_h160 = decode_base58(my_address)
# Script_pubkeyを生成する。
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_satoshi, script_pubkey=target_script)
# 配列に追加する。
tx_outs.append(change_output)
#  トランザクションデータを生成する。
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# 1つ目のインプットトランザクションに対して署名する。
print(tx_obj.sign_input(0, priv))
# 2つ目のインプットトランザクションに対して署名する。
print(tx_obj.sign_input(1, priv2))
# 16進数でトランザクションを出力する。
print(tx_obj.serialize().hex())

False
False
